## Открываем файл с данными и изучаем общую информацию

In [76]:
import pandas as pd
df = pd.read_csv('/datasets/data.csv')
display(df.head(10))
df.info()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**Вывод**

Первое, что бросается в глаза, это отрицательные и пропущенные значения в days_employed. days_employed(трудовой стаж) в некоторых ячейках указан в отрицательном диапазоне. А где указан положительно, то указан, видимо не в днях, а в час, может быть. Потому что, к примеру, клиент банка под номером 4 имеет стаж 340266 дней, это 932 года, а 340266 часов это почти 39 лет. А вот значения, со знаком минус, как раз, дни. 

Но лучше уточнять такие вопросы у тех, кто данные предоставляет. Нулевые значения встречаются, по всей видимости, по причине не внимательного заполнения данных менеджерами банка при обработке заявки на кредит.

В столбце total_income тоже встречаются нулевые значения. Это. скорее всего, тоже ошибка при заполнении данных менеджерами банка. Человеческий фактор. Но при всех "косяках" в данных с ними можно работать.

Еще, строковые значения в некоторых столбцах заполнены разынм регистром. Но эта проблема легко решается при предобработке данных (приводим все к одному регистру).

## Предобработка данных

### Обработка пропусков

In [77]:
df.isna().sum() #считаем, сколько есть пропусков и где они

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Получается, что в столбцах days_employed и total_income по 2174 пропуска в каждом, что было видно при вызове функции df.info(). Пропуски, как мне кажется, логичнее заменить медианой. Но чтобы это сделать, нужно снала привести столбец days_employed к показателю дни, и чтобы не былот отрицательдных значений.

Возможно, пропуски появились по причине человеческого фактора. Ведь информацию о клиентах банка заполняют банковские менеджеры, а люди часто ошибаются.

In [78]:
def day(i): #пишем функцию, которая приводит к дням
    if i>0: #если значение в days_employed положительное, значит указано в часах.
        return i/24 #часы переводим в дни
    return -i #если значение в days_employed, значит указано в днях, нужно просто убрать знак минус
df['days_employed']=df['days_employed'].apply(day) #редактируем столбец days_employed, применив к каждому значению столбца ф-ю day
display(df.head(10)) #проверям: работает

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,14177.753002,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [79]:
df['days_employed']=df['days_employed'].fillna(df['days_employed'].median())# заменяем Nan на медиану в стаже и уровне дохода
df['total_income']=df['total_income'].fillna(df['total_income'].median())
df.info()#проверяем

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**Вывод**


Я привел значени столбца days_employed к дням. После чего, заменил пропуски в столбцах days_employed и total_income на медианное значение этих столбцов.

### Замена типа данных

In [80]:
df['days_employed'] = df['days_employed'].astype('int')#меняем тип данных столбцах days_employed и total_income с float64 на int
df['total_income'] = df['total_income'].astype('int')
df['dob_years'] = df['dob_years'].astype('int')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


**Вывод**

В столбцах days_employed и total_income поменял тип данных вещественных на целочисленные, при помощи метода astype()

### Обработка дубликатов

In [81]:
df['education'] = df['education'].str.lower() #приведем все столбцы, в которых есть текст, к нижнему регистру.
df['income_type'] = df['income_type'].str.lower()
df['purpose'] = df['purpose'].str.lower()
df['gender'] = df['gender'].str.lower()
df['family_status'] = df['family_status'].str.lower()
df.duplicated().sum() #и посчитает кол-во дубликатов.

71

Дубликатов 71 строка. Например, возраст, образование, .По отношению ко всей таблице, это 0,33% от всех данных. Это незначительно количество можно удалить без вреда для исследования.

In [82]:
duplicated=df[df.duplicated()].head()
display(duplicated)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
2849,0,2194,41,среднее,1,женат / замужем,0,f,сотрудник,0,145017,покупка жилья для семьи
3290,0,2194,58,среднее,1,гражданский брак,1,f,пенсионер,0,145017,сыграть свадьбу
4182,1,2194,34,высшее,0,гражданский брак,1,f,сотрудник,0,145017,свадьба
4851,0,2194,60,среднее,1,гражданский брак,1,f,пенсионер,0,145017,свадьба
5557,0,2194,58,среднее,1,гражданский брак,1,f,пенсионер,0,145017,сыграть свадьбу


В этих строка не все значения одинаковые. Например возраст, кол-во детей у некоторых строк отличается. Но, все равно, примем за дубликаты, потому что на общую картину это не повлияет.

Дубликаты появились по нескольким причинам. Одна из них заполнение пропущеных значение в столбцах медианой. Вторая причина - повторное обращение одного и того же клиента в банк - сделали две одинакове заявки, но изменили причину, например, со "свадьба" на "сыграть свадьбу".

In [83]:
df = df.drop_duplicates().reset_index(drop=True) #удаляем дубликаты с обновлением индексации
df.duplicated().sum() #проверяем кол-во дубликатов после их удаления

0

**Вывод**

Я посчитал количество дубликатов при помощи метода duplicated().sum() и удалил их при помощи метода drop_duplicates().reset_index(drop=True) с обновлением индексации и удалением старой.

Перед этим, привел все значения в стоблцах, которые заполнены строковыми величинами, к нижему регистру. Сделал для всех столбцов, где данные представлены строковыми значениям. Вдруг, где-то в середине датафрейма, где я не вижу, написано ЗаБоРчИкОм:) После этого, посчитал количество дубликтов. 

Далее я обнаружил, что найденные строки-дубликаты одинаковы не по всем значениям(возраст, количество детей). Но посчитав отношение этих строк к общему датафрейму(0,33%), решил их удалить, ведь их удаление не повредит результату исследования, по причине своего маленького количества.

### Лемматизация

In [84]:
from pymystem3 import Mystem
m = Mystem()
reasons = df['purpose'].unique()
s=''
for text in reasons:
    l = ''.join(m.lemmatize(text))
    s+=(l)
print(s)

покупка жилье
приобретение автомобиль
дополнительный образование
сыграть свадьба
операция с жилье
образование
на проведение свадьба
покупка жилье для семья
покупка недвижимость
покупка коммерческий недвижимость
покупка жилой недвижимость
строительство собственный недвижимость
недвижимость
строительство недвижимость
на покупка подержать автомобиль
на покупка свой автомобиль
операция с коммерческий недвижимость
строительство жилой недвижимость
жилье
операция со свой недвижимость
автомобиль
заниматься образование
сделка с подержанный автомобиль
получение образование
автомобиль
свадьба
получение дополнительный образование
покупка свой жилье
операция с недвижимость
получение высокий образование
свой автомобиль
сделка с автомобиль
профильный образование
высокий образование
покупка жилье для сдача
на покупка автомобиль
ремонт жилье
заниматься высокий образование



**Вывод**

Проведя лемматизацию, я выделил четыре основные причины кредитования по ключевым словам: "автомобиль", "свадьба", "недвижимость"("жилье") и "образование".

### Категоризация данных

**2.5.1 Зависимость между наличием детей и возвратом кредита в срок**

In [85]:
def children(child): #функция, для категоризации по кол-ву детей. где:
    if child==0:
        return 1 # 1- нет детей
    if 0<child<=2:
        return 2 # 2- 1-2 ребенка
    return 3     # 3- 3 и более детей

In [86]:
#dict_children = df[['children', 'debt', 'total_income']]#делаем датафрейм из трех столбцов исходного
#none_children = dict_children[dict_children['children']==0]
#none_children = none_children[none_children['debt']!=0]
#more_none_children = dict_children[dict_children['children']!=0]
#more_none_children = more_none_children[more_none_children['debt']!=0]
#print('Количество заемщиков с детьми, имевеших задоженности по кредитам:', len(more_none_children))
#print('Количество заемщиков без детей, имевеших задоженности по кредитам:', len(none_children))

In [87]:
dict_children1 = df[['children', 'debt']] #добавляем столбец с категорией по кол-ву детей
dict_children1.loc[:,'child_level'] = dict_children1.loc[:, 'children'].apply(children)
dict_children2=dict_children1[['child_level', 'debt']]
count_of_children2 = dict_children2.pivot_table(index='child_level', 
                                      aggfunc=['count', 'sum'])#делаем сводную таблицу
count_of_children2['percentage_of_debt'] = (count_of_children2[('sum','debt')]*100)/count_of_children2[('count','debt')]
# % людей с задолженностями

display(count_of_children2)

,count,sum,percentage_of_debt
,debt,debt,
child_level,,,
1,14091,1063,7.543822
2,6860,638,9.300292
3,503,40,7.952286


**2.5.2 Зависимость между семейным положением и возвратом кредита в срок**

In [88]:
dict_family = df[['family_status', 'family_status_id', 'debt']]#делаем датафрейм из трех столбцов исходного
count_of_id = dict_family.pivot_table(index=['family_status', 'family_status_id', ], 
                                      aggfunc=['count', 'sum'])#делаем сводную таблицу
count_of_id['percentage_of_debt'] = (count_of_id[('sum','debt')]*100)/count_of_id[('count','debt')]# % людей с задолженностями

count_of_id = count_of_id.sort_values(by=('percentage_of_debt'))# сортировка по % задоженностей
display(count_of_id)

,,count,sum,percentage_of_debt
,,debt,debt,
family_status,family_status_id,,,
вдовец / вдова,2,959,63,6.569343
в разводе,3,1195,85,7.112971
женат / замужем,0,12339,931,7.545182
гражданский брак,1,4151,388,9.347145
не женат / не замужем,4,2810,274,9.750890


**Вывод**

'count debt' - количество человек, соответствующее статусу в столбце 'family_status'.

'sum debt' - количество человек, имевших задолженности по выплате кредитов.

'percentage_of_debt' - процентное отношение 'count debt' к 'sum debt' в %, где 'count debt' это 100%. 

**2.5.3 Зависимость между уровнем дохода и возвратом кредита в срок**

Для ответа на этот вопрос необходимо сделать словарь по уровню дохода. Средняя зарплата в России в августе 2021 года, согласно Росстату, составляет 52 тыс. руб. Округлим до 50 тыс. руб.

На основании этого, выделим три основные группы:

low - уровень дохода до 50 тыс. руб. включительно

middle - 51-150 тыс. руб.

high - >150 тыс. руб.

Для создания класса по уровню дохода испрользуем фунцию salary(money)

In [89]:
def salary(money): #функция, для категоризации по уровню дохода.
    if money<=50000:
        return 'low'    #low - уровень дохода до 50 тыс. руб. включительно
    if 51000<money<=150000:
        return 'middle' #middle - 51-150 тыс. руб.
    return 'high'       #high - >150 тыс. руб.

In [90]:
dict_salary = df[['total_income', 'debt']]#делаем датафрейм из двух столбцов исходного
dict_salary.loc[:,'salary_level'] = dict_salary.loc[:,'total_income'].apply(salary)#создаем столбец 'salary_level' и применяем к нему 
#функцию salary
display(dict_salary.head())#проверяем. конуструкцию loc применил для избежания ошибки  value is trying 
#to be set on a copy of a slice from a DataFrame.

,total_income,debt,salary_level
0,253875,0,high
1,112080,0,middle
2,145885,0,middle
3,267628,0,high
4,158616,0,high


In [91]:
dict_salary['salary_level'].value_counts() #cчитаем количество человек в группах

middle    11862
high       9220
low         372
Name: salary_level, dtype: int64

In [92]:
dict_salary1=dict_salary[['debt', 'salary_level']]#создаем новую таблицу, уже без total_income, чтобы создать читаемую
count_of_salary = dict_salary1.pivot_table(index=['salary_level'], #сводную таблицу
                                      aggfunc=['count', 'sum'])
count_of_salary['percentage_of_debt'] = (count_of_salary[('sum','debt')]*100)/count_of_salary[('count','debt')]
display(count_of_salary)

,count,sum,percentage_of_debt
,debt,debt,
salary_level,,,
high,9220,729,7.906725
low,372,23,6.182796
middle,11862,989,8.337548


**2.5.4 Как разные цели кредита влияют на его возврат в срок**

In [93]:
purpose_stem=["автомобиль", "свадьба", "недвижимость", "жилье", "образование" ]
dict_purpose= df[['debt', 'purpose']]#создаем таблицу на основании столбцов 'debt', 'purpose'
display(dict_purpose.head())

,debt,purpose
0,0,покупка жилья
1,0,приобретение автомобиля
2,0,покупка жилья
3,0,дополнительное образование
4,0,сыграть свадьбу


In [94]:
def lemm(x): #использую лемматизацию для того, чтобы привести причины для взятия кредита к исходным словам. чтобы объеденить их
    l=m.lemmatize(x)
    return l
dict_purpose.loc[:,'purpose1'] = dict_purpose.loc[:, 'purpose'].apply(lemm)
display(dict_purpose.head())

,debt,purpose,purpose1
0,0,покупка жилья,"[покупка, , жилье, \n]"
1,0,приобретение автомобиля,"[приобретение, , автомобиль, \n]"
2,0,покупка жилья,"[покупка, , жилье, \n]"
3,0,дополнительное образование,"[дополнительный, , образование, \n]"
4,0,сыграть свадьбу,"[сыграть, , свадьба, \n]"


In [95]:
for k in range(len(dict_purpose)): #заменяю причины на группы  (проведение свадьбы->свадьба)
    for i in purpose_stem:
        if i in dict_purpose.loc[k, 'purpose1']:
            dict_purpose.loc[k,'purpose2']=i
for k in range(len(dict_purpose)):
    if dict_purpose.loc[k, 'purpose2']=='жилье': #объединяю недвижимость и жилье
        dict_purpose.loc[k, 'purpose2']='недвижимость'
dict_purpose=dict_purpose[['debt', 'purpose2']] #делаю обновленную таблицу и считаю количество причин кредита
dict_purpose['purpose2'].value_counts()
count_of_purpose = dict_purpose.pivot_table(index=['purpose2'], #сводная таблица
                                      aggfunc=['count', 'sum'])
count_of_purpose['percentage_of_debt'] = (count_of_purpose[('sum','debt')]*100)/count_of_purpose[('count','debt')]
display(count_of_purpose)

,count,sum,percentage_of_debt
,debt,debt,
purpose2,,,
автомобиль,4306,403,9.359034
недвижимость,10811,782,7.233373
образование,4013,370,9.220035
свадьба,2324,186,8.003442


## Ответы на вопросы исследования

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [96]:
display(count_of_children2)

,count,sum,percentage_of_debt
,debt,debt,
child_level,,,
1,14091,1063,7.543822
2,6860,638,9.300292
3,503,40,7.952286


**Вывод**

'count debt' - количество человек с определенным кол-вом детей.

'sum debt' - количество человек, имевших задолженности по выплате кредитов.

'percentage_of_debt' - процентное отношение 'count debt' к 'sum debt' в %, где 'count debt' это 100%.

child_level:

1 - нет детей,

2 - 1-2 ребенка,

3 - 3 и более детей.

На основании провденного исследования, можно заметить, что:

1) Если у кредитора нет детей, то вероятность просрочки по платежу меньше всего.

2) Если у кредитора не более 2х детей, то вертояность просрочки самая большая.

3) Если у кредитора 3 и более детей, то вертоятность просрочки почти такая же, как у кредиторов без детей.

Эти выводы отличаются от моих предыдущих по этому вопросу. Я понял, что граммотная категоризация данных, это очень важно:)

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [98]:
display(count_of_id)

,,count,sum,percentage_of_debt
,,debt,debt,
family_status,family_status_id,,,
вдовец / вдова,2,959,63,6.569343
в разводе,3,1195,85,7.112971
женат / замужем,0,12339,931,7.545182
гражданский брак,1,4151,388,9.347145
не женат / не замужем,4,2810,274,9.750890


**Вывод**

'count debt' - количество человек, соответствующее статусу в столбце 'family_status'.

'sum debt' - количество человек, имевших задолженности по выплате кредитов.

'percentage_of_debt' - процентное отношение 'count debt' к 'sum debt' в %, где 'count debt' это 100%. 

По сводной таблице видно, какой id какому семейному статусу соответсвует, и их количество.

Также видно, у скольких из них были задоженности по кредитам, а в столбце "percentage_of_debt" отображен процент людей, имевших задолженности.

Сортировка таблицы по проценту людей с задолженностями по кредитам.

Заметно, что у людей вдовцов и в разводе, процент по задолженностям меньше, чем у женатых или холостых людей. Вообще, можно поделить людей на две категории: В браке и нет. И если выводить среднее значение задолженностей по кредитам людей в браке, то будет: 

**(percentage_of_debt(женат / замужем)+percentage_of_debt(гражданский брак))/2=8,45%** - женатые 

**(percentage_of_debt(вдовец / вдова)+percentage_of_debt(в разводе)+percentage_of_debt(не женат / не замужем))/3=8,16%** - одинокие

Из этого делаю вывод: зависимости между семейным положением и возвратом кредита в срок нет.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [75]:
display(count_of_salary)

,count,sum,percentage_of_debt
,debt,debt,
salary_level,,,
high,9220,729,7.906725
low,372,23,6.182796
middle,11862,989,8.337548


**Вывод**

'count debt' - количество человек, соответствующее статусу в столбце 'salary_level'.

'sum debt' - количество человек, имевших задолженности по выплате кредитов.

'percentage_of_debt' - процентное отношение 'count debt' к 'sum debt' в %, где 'count debt' это 100%.

Аналогично подсчету в разделе "семейное положение", считаем процентоное отношение людей, имевших задолженность по выплатам.
Интересно получилось, у людей с уровнем дохода из низкой категории процент задолжнности самый маленький.

Разница между max и min значением всего 1,2%. Поэтому можно сделать вывод, что уровень дохода немного влияет на возврат кредита в срок.

- Как разные цели кредита влияют на его возврат в срок?

Из категорий целей кридита, п. 2.4, выделяем основы слов при помощи стемминга. Это нужно для того, чтобы можно было отнести, к примеру, "сыграть свадьбу" и "свадьба" к одной категории, а не к разным.

In [99]:
display(count_of_purpose)

,count,sum,percentage_of_debt
,debt,debt,
purpose2,,,
автомобиль,4306,403,9.359034
недвижимость,10811,782,7.233373
образование,4013,370,9.220035
свадьба,2324,186,8.003442


**Вывод**

'count debt' - количество причин.

'sum debt' - количество человек, имевших задолженности по выплате кредитов.

'percentage_of_debt' - процентное отношение 'count debt' к 'sum debt' в %, где 'count debt' это 100%.

Аналогично предыдущим выводам, выводим процент должников по причинам кредита. Здесь уже разница между max и min 2,1%.

Больше всего просрочек у тех, кто берет кредит на автомобиль или образование.

Реже просрачивают платежи те, кто берет на свадьбу.

Меньше всех просрачивают те, что берет кредит на преобретение или ремонт недвижимости.

Из этого делаю вывод, разные цели кредита немного, но влияют на его возврат в срок.

## Общий вывод

Полученные данные содержали некоторые артефакты.

days_employed(трудовой стаж) в некоторых ячейках указан в отрицательном диапазоне. А где указан положительно, то указан, видимо не в днях, а в час. Потому что, к примеру, клиент банка под номером 4 имеет стаж 340266 дней, это 932 года, а 340266 часов это почти 39 лет. А вот значения, со знаком минус, как раз, дни. Я конвертировал все к одному виду - к дням. 

Далее, в столбце education(образование) данные заполнены в разном регистре. Все столобцы, которые содержат строковые данные,  я привел к нижнему регистру, для корректной работы над ними.

Затем, обработал пропуски, столбцах days_employed и btotal_income. Заполнил их медианой этих столбцов. Заменил тип данных с float на int.

Затем принялся за обработку дубликатов. Их было не много, всего 0,33% от всех данных. Но, как верно было замечено, от дубликатов нужно избавляться вне зависимости от их количества:)
    
Сделал лемматизацию в колонке с целью кредита, чтобы дальше можно было сгруппировать кредиторов по целям.
    
Далее, я категоризировал кредиторов по тем вопросам, на которые просил ответить заказчик: 

- наличие детей,
    
- семейное положение,
    
- уровень дохода,
    
- цель кредита.

После группировки, я посчитал количество кредиторов в каждой группе имевших задолженности по кредитам, и не имевших задолженности. Это и стало ключевым фактором в ислледовании. Посчитал процентное соотношение должников к не должникам, и опирался на эти данные чтобы сделать выводы по каждому из вопросов:
    
1) Наличие детей влияет на возврат кредита в срок. 

2) Семейное положение не влияет на возврат кредита в срок. Так как статусов семейного положения целых 5, я объеденил их в 2. Посчитав средний процент должников в каждой категории, я обнаружил, что они почти одинаковые (разница 0,3%).

3) Уровень дохода немного, но влияет возвратом кредита в срок. В процентом соотношении разница совсем небольшая - 1,2%. Причем, чем выше уровень дохода, тем меньше вероятность возврат кредита в срок.

4) Разные цели кредита тоже немного, но влияют на возврат в срок. Больше всего просрочек у тех, кто берет кредит на автомобиль или образование. Реже просрачивают платежи те, кто берет на свадьбу. Меньше всех просрачивают те, что берет кредит на приобретение или ремонт недвижимости. 
    
Рекомендация заказчику.
    
На основании предоставленных данных, можно сделать модель идеального кредитора - проверить все столбцы на просрочку платежей, и вывести, при каких условиях вероятность просрочки будет 0%, а при каких 100%.